add data

In [ ]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=4000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
vocabulary = list()
for text in x_train:
  tokens = text.split()
  vocabulary.extend(tokens)

vocabulary = set(vocabulary)
print(len(vocabulary))

3998


In [ ]:
from tqdm import tqdm

x_train_binary = list()
x_test_binary = list()

for text in tqdm(x_train):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_train_binary.append(binary_vector)

x_train_binary = np.array(x_train_binary)

for text in tqdm(x_test):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_test_binary.append(binary_vector)

x_test_binary = np.array(x_test_binary)

100%|██████████| 25000/25000 [07:10<00:00, 58.04it/s]


In [ ]:
print(x_test_binary[2])
print(y_test[2])
print(x_train_binary[2])
print(y_train[2])

[1 0 0 ... 0 0 0]
1
[0 0 0 ... 0 0 0]
0


Calculate entropy

In [ ]:
def calculate_entropy_word(binary_array, result_array, c, word, word_position, cProb):
  if(cProb == 1 or cProb == 0):
    return 0
  return - ( cProb * np.log2(cProb) ) - ( (1.0 - cProb) * np.log2(1.0 - cProb))


def calculate_entropy_total(binary_array, result_array):
  first = 0 # c value
  count = 0 # lengeth
  for element in binary_array:
      if(result_array[count] == 1):
        first += 1
      count += 1
  cProb = first/binary_array.shape[0]
  if(cProb == 1 or cProb == 0):
    return 0      
  return - ( cProb * np.log2(cProb) ) - ( (1.0 - cProb) * np.log2(1.0 - cProb))

  

#print(calculate_entropy_total(x_train_binary, y_train))  
#print(calculate_entropy_word(x_train_binary,y_train, 1, Word(None, 0), 0))
print(len(y_train))

25000


return the position of a word

In [ ]:
# -1 = has not found the word
def return_word(word):
  count = 0
  for element in vocabulary:
    if word == element:
      return count
    else:
      count += 1
  
  return -1

calculate probability

In [ ]:
def calculate_probability(array, result_array, c, word, word_position = -1):
  #Entropy
  has_the_word = 0 # c value
  count = 0 # lengeth
  is_positive = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  
  for element in array:
      element_i = element[word_position]
      count += 1
      if(element_i == c):
        has_the_word += 1
        if(result_array[count-1] == 1):
          is_positive += 1
  
  if(has_the_word == 0):
    #ToDo DES TO
    return 0
  
  return is_positive/has_the_word

In [ ]:
def calculate_probability_total(result_array):
  #Node 
  count = 0
  p = 0
  for x in result_array:
    if(x == 1):
      p += 1
    count += 1
  return p/count

calculate sum

In [ ]:
def calculate_sum(binary_array, result_array, word):
    word_position = word.index
    p0 = (calculate_probability(binary_array, result_array, 0, word, word_position))
    p1 = 1 - p0
    word.setEntropy0(calculate_entropy_word(binary_array, result_array, 0, word,  word_position, p0))
    word.setEntropy1(calculate_entropy_word(binary_array, result_array, 1, word,  word_position, p1))
    if p0 == 0 or word.entropy0 ==0:
      a = 0
    else:
      a = np.log2(p0 * word.entropy0)

    if p1 == 0 or word.entropy1 ==0:
      b = 0
    else:
      b = np.log2(p1 * word.entropy1)
    return  a + b

calculate IG

In [ ]:
def calculate_IG(binary_array, result_array, word, calculate_entrupy_total):
    word.setIg(calculate_entrupy_total - calculate_sum(binary_array, result_array, word))

In [ ]:
class Word:

  def __init__(self, value:str, index:int):
    self.value = value
    self.index = index
    self.ig = None
    self.entropy0 = None
    self.entropy1 = None
  
  def setIg(self, ig):
    if(ig != None):
      self.ig = float(ig)
      
  def getIg(self):
    return self.ig

  def setEntropy0(self, entropy0):
    self.entropy0 = entropy0

  def setEntropy1(self, entropy1):
    self.entropy1  = entropy1

  def __str__(self):
    return self.value

  def __repr__(self):
    return self.value + " " + str(self.ig)
  

Node

In [ ]:
def hasTheWord(word1: Word, word2: Word, array):
  index1 = int(word1.index)
  index2 = int(word2.index)
  for x in array:
    if(x[index1] == 1 and x[index2] == 1):
      return 1
  return 0

In [ ]:
def remove_words(array, word_index, keep_the_word = True):
  num = []
  c = 1
  if(keep_the_word):
    c = 0
  count = 0
  for x in array:
    if(x[word_index] == c):
      num.append(count)
    count+=1

  return num #np.delete(array, num, 0)

print(len(remove_words(x_train_binary, 3730, False)))
print(len(remove_words(x_train_binary, 3730)))

169
24831


In [ ]:
class Node:
  value: Word
  left = None # must be Node
  right = None # must be Node
  depth = None
  p0 = None
  p1 = None

  def __init__(self, v: int = None, depth = None):
    self.value = v
    if depth == None:
      self.depth = 10
    else:
      self.depth = depth
  
  def setLeftChild(self, v: Word):
    self.left = v

  def setRightChild(self, v: Word):
    self.right  = v

  def setChilds(self, l, r):
    self.setLeftChild(l)
    self.setRightChild(r)

  def setP0(self, p0):
    self.p0 = p0
  
  def setP1(self, p1):
    self.p1 = p1

  def __str__(self):
    return self.value.value

Remove words

In [ ]:
#there is the word in the setence
def findValueFromArray(array, index):
  if len(array) < index:
    return None
  try:
    if(array[index] == 1):
     return index
  except:
    return None
  return None

In [ ]:
import math 
def get_my_key(obj:Word):
  return obj.getIg()

def next_word_subtree(array, result_array, words):
  i = 0
  et = calculate_entropy_total(array, result_array)
  words_to_remove = list()
  for word in words:
   word.setEntropy0(None)
   word.setEntropy1(None)
   word.setIg(None)
   calculate_IG(array, result_array, word, et)
  for x in words:
    if x.ig == None or math.isnan(x.ig):
     words_to_remove.append(x)
  for x in words_to_remove:
      words.remove(x)
  #words.sort(reverse=True, key=get_my_key)
  words = sorted(words, key=get_my_key, reverse=True)
  re = words.pop()
  return (re, words)



In [ ]:
words = calculate_words()
print(len(x_train_binary))
print(len(words))
data = next_word_subtree(x_train_binary, y_train, words)
print(data)

25000
3996
(alone 3.0, [boll 12.737239994004213, incoherent 9.365938844119018, mst3k 8.76252870308834, seagal 8.29989492846237, unfunny 8.112723194034194, waste 7.704116234415141, flawless 7.691998717324379, matthau 7.523639088575765, horrid 7.2869982682587695, superbly 7.256364788928487, atrocious 7.256364788928487, pointless 7.115407490432135, blah 7.053377903168913, redeeming 7.042941679897531, drivel 6.941872543273921, perfection 6.805029807511074, laughable 6.7325273360263775, lousy 6.690830551087513, worst 6.680129652515021, wasting 6.61529526814544, remotely 6.555186212396595, awful 6.4890979318827675, poorly 6.408278417057342, captures 6.25929737613413, existent 6.221910890416405, wonderfully 6.201765812216544, insult 6.058613202234289, boredom 6.019382552377046, sucks 6.013597277199288, miserably 5.977343158536796, uninspired 5.977343158536796, lame 5.975739060561352, stupidity 5.910606199564269, refreshing 5.904833168094241, breathtaking 5.858264219868793, amateurish 5.825071

In [ ]:
def calculate_words():
  words = list()
  i = 0
  for x in vocabulary:
    if(x != '[pad]' and x!= '[bos]' and x != '[oov]'):
      words.append(Word(x, i))
    i+= 1

  return words

In [ ]:
def main(tree_result):
  good_result = 0
  count = 0
  for sentense in x_train_binary:
    tree = tree_result
    result = None
    while True:
      if tree.left == None and tree.right == None or tree.p1 == 1 or tree.p0 == 1:
        result = tree
        break
      else:
        sentense_word = findValueFromArray(sentense, tree.value.index)
        if(sentense_word == None):
          if tree.left == None:
            result = tree
            break
          tree = tree.left
        else:
          if tree.right == None :
            result = tree
            break 
          tree = tree.right
    c = 1
    if(result.p0 > result.p1):
      c = 0
    if(y_train[count] == c):
      good_result += 1
    count += 1
  return good_result/count*100

In [34]:
def fit(tree_result, limit: int): #fit
  good_result = 0
  count = 0
  check = x_train_binary[:limit]
  for sentense in check:
    tree = tree_result
    result = None
    while True:
      if tree.left == None and tree.right == None or tree.p1 == 1 or tree.p0 == 1:
        result = tree
        break
      else:
        sentense_word = findValueFromArray(sentense, tree.value.index)
        if(sentense_word == None):
          if tree.left == None:
            result = tree
            break
          tree = tree.left
        else:
          if tree.right == None :
            result = tree
            break 
          tree = tree.right
    c = 1
    if(result.p0 > result.p1):
      c = 0
    if(y_train[count] == c):
      good_result += 1
    count += 1
  return good_result/count*100

In [ ]:
def main_test(tree_result): #predict
  good_result = 0
  count = 0
  for sentense in x_test_binary:
    tree = tree_result
    result = None
    while True:
      if tree.left == None and tree.right == None or tree.p1 == 1 or tree.p0 == 1:
        result = tree
        break
      else:
        sentense_word = findValueFromArray(sentense, tree.value.index)
        if(sentense_word == None):
          if tree.left == None:
            result = tree
            break
          tree = tree.left
        else:
          if tree.right == None :
            result = tree
            break 
          tree = tree.right
    c = 1
    if(result.p0 > result.p1):
      c = 0
    if(y_test[count] == c):
      good_result += 1
    count += 1
  return good_result/count*100

In [40]:
def predict(tree_result, limit: int):
  good_result = 0
  count = 0
  check = x_test_binary[:limit]
  ar_result = y_test[:limit]
  for sentense in check:
    tree = tree_result
    result = None
    while True:
      if tree.left == None and tree.right == None or tree.p1 == 1 or tree.p0 == 1:
        result = tree
        break
      else:
        sentense_word = findValueFromArray(sentense, tree.value.index)
        if(sentense_word == None):
          if tree.left == None:
            result = tree
            break
          tree = tree.left
        else:
          if tree.right == None :
            result = tree
            break 
          tree = tree.right
    c = 1
    if(result.p0 > result.p1):
      c = 0
    if(ar_result[count] == c):
      good_result += 1
    count += 1
  return good_result/count*100

In [ ]:
def anadromi(array, result_array, depth, words = None):
  if(words == None):
    words = calculate_words()
  if(len(array) == 0 or len(words) == 0 or depth == 0):
    return None
  data = next_word_subtree(array, result_array, words)
  word = data[0]
  words = data[1]
  node = Node(word, depth)
  node.setP1(calculate_probability_total(result_array))
  node.setP0(1-node.p1)
  l_num = remove_words(array, word.index, False)
  l_array = np.delete(array, l_num, 0)
  l_result_array = np.delete(result_array, l_num, 0)
  node.setLeftChild(anadromi(l_array, l_result_array, depth-1, words.copy()))
  r_num = remove_words(array, word.index)
  r_array = np.delete(array, r_num, 0)
  r_result_array = np.delete(result_array, r_num)
  node.setRightChild(anadromi(r_array, r_result_array, depth-1, words.copy()))
  return node

In [42]:
x_copy = x_train_binary[:100]
y_copy =y_train[:100]
tree = anadromi(x_copy, y_copy, 300)
print("train")
print(main(tree))
print("test")
print(main_test(tree))
print("train limit:")
print(main_limit(tree,10))
print(main_limit(tree,20))
print(main_limit(tree,30))
print(main_limit(tree,40))
print(main_limit(tree,50))
print(main_limit(tree,100))
print(main_limit(tree,200))
print(main_limit(tree,300))
print("test limit:")
print(main_test_limit(tree,10))
print(main_test_limit(tree,20))
print(main_test_limit(tree,30))
print(main_test_limit(tree,40))
print(main_test_limit(tree,50))
print(main_test_limit(tree,100))
print(main_test_limit(tree,200))
print(main_test_limit(tree,300))

train
54.064
test
53.748
train limit:
100.0
100.0
100.0
100.0
100.0
100.0
75.0
67.33333333333333
test limit:
40.0
55.00000000000001
60.0
60.0
64.0
63.0
57.99999999999999
58.666666666666664
